## TODOs:
* Find way to get positive reward
* Use tf.abs() to calc logp
* Discover why NN always return y=1.0

In [7]:
import kagglegym

# Create environment
env = kagglegym.make()
# Get first observation
observation = env.reset()

observation.features.shape

(968, 110)

In [8]:
# hyperparameters
batch_size = 5 # every how many episodes to do a param update?
learning_rate = 1e-2 # feel free to play with this to train faster or more stably.
gamma = 0.99 # discount factor for reward

D = observation.features.head().shape[1] # input dimensionality
H = int(D / 3) # number of hidden layer neurons

[D, H]

[110, 36]

In [9]:
import tensorflow as tf

tf.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action of moving left or right.
observations = tf.placeholder(tf.float32, [None,D] , name="input_x")
W1 = tf.get_variable("W1", shape=[D, H],
           initializer=tf.contrib.layers.xavier_initializer())
W1 = tf.Print(W1, [W1], message="W1=")
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W2 = tf.get_variable("W2", shape=[H, 1],
           initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.Print(W2, [W2], message="W2=")
score = tf.matmul(layer1,W2)
probability = tf.nn.sigmoid(score)

#From here we define the parts of the network needed for learning a good policy.
tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.placeholder(tf.float32,name="reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
#TODO fix loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
#loglik = tf.log(tf.abs(input_y - probability))
#TODO fix loss = -tf.reduce_mean(loglik * advantages) 
loss = -tf.reduce_mean((input_y - probability) * advantages)
newGrads = tf.gradients(loss,tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
# Placeholders to send the final gradients through when we update.
W1Grad = tf.placeholder(tf.float32,name="batch_grad1") 
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
#
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))


In [10]:
import numpy as np

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    # in Python 2:
    # for t in reversed(xrange(0, r.size)):
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r


In [11]:
class Debug:
    def __init__(self, state):
        self.state = state

    def log(self, msg):
        if self.state % 300 == 0:
            print(msg)


In [28]:
# Here's an example of loading the CSV using Pandas's built-in HDF5 support:
import pandas as pd

with pd.HDFStore("../input/train.h5", "r") as train:
    # Note that the "train" dataframe is the only dataframe in the file
    df = train.get("train")

print(df[df["timestamp"] == 909].shape)
print(df[df["timestamp"] == 1000].shape)
print(df[df["timestamp"] == 1100].shape)

t = df[df["timestamp"] == 1812]
print(t.shape)
t.head(10)



(968, 111)
(959, 111)
(974, 111)
(1086, 111)


,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
1709670,0,1812,0.031061,-0.050150,-0.003554,0.069194,-0.487231,-0.074018,0.367250,-0.104078,...,0.181371,-1.183291e-29,-8.574926e-26,-7.006492e-45,-0.141797,0.155133,1.040918e-10,-4.279235e-09,-0.028572,-0.001499
1709671,6,1812,-1.344294,-0.021178,0.018444,-0.023027,0.077929,-0.460082,NaN,0.272517,...,0.338830,-1.100825e-10,0.000000e+00,-1.893266e-28,-0.129603,-0.103904,-1.100812e-10,-2.328306e-10,0.005868,-0.006842
1709672,7,1812,-0.895752,0.261903,1.302218,-0.298881,0.183958,-0.364517,-0.115767,0.154581,...,-0.298378,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.043851,-0.182204,0.000000e+00,-3.330669e-16,0.017469,-0.017847
1709673,11,1812,0.032811,-0.047328,0.342012,0.220401,0.146986,0.201177,0.085757,0.011716,...,0.261470,-2.023203e-09,-2.023202e-09,-2.536633e-33,-0.175305,-0.194621,-8.911819e-01,-1.660443e-06,0.004676,-0.001772
1709674,12,1812,-0.097520,-0.061873,-0.078045,-0.103562,-0.100693,-0.220676,0.155560,0.092945,...,0.293509,-1.272979e-15,-3.547893e-04,-1.111081e-39,-0.064990,0.049166,2.023202e-09,-1.266820e-11,0.000315,-0.004372
1709675,13,1812,-0.025554,-0.046826,NaN,-0.026069,NaN,-0.049894,0.046646,NaN,...,0.090820,-0.000000e+00,-0.000000e+00,0.000000e+00,-0.133357,0.041934,-0.000000e+00,-3.330669e-16,0.001117,0.008202
1709676,14,1812,0.124100,-0.003981,NaN,0.016947,NaN,-0.456786,-0.397963,NaN,...,0.665351,0.000000e+00,-7.850498e-07,-7.850498e-07,0.521092,-0.005332,0.000000e+00,-2.328306e-10,-0.004904,-0.015128
1709677,15,1812,-0.632553,0.537024,NaN,-0.209763,NaN,-0.144593,-0.400324,NaN,...,1.094934,-2.023202e-09,-3.547893e-04,-0.000000e+00,0.610119,-0.059092,1.036645e-01,-1.266820e-11,0.045099,-0.001641
1709678,16,1812,-1.057133,1.693776,0.327569,-0.350504,-1.058410,0.096709,-0.425292,-0.689071,...,3.047898,-6.115252e-22,-1.100812e-10,-1.183291e-29,0.676257,0.132892,-9.897066e-01,-2.328306e-10,0.028043,0.005575
1709679,17,1812,0.066642,0.011794,0.102463,0.278758,0.031048,-0.036045,NaN,-0.084615,...,0.519884,-1.036645e-01,-7.851599e-07,-7.851599e-07,-0.080266,-0.106280,-9.948465e-01,-1.266820e-11,0.012489,-0.013945


g W2 6275.03595817                                                      [7/1817]
model W2 -0.0826594463929
episode reward total was -10724.000000. running mean: -9499.148827
Episode 270, Step 880: -1.00 -1.00 -1.00 -1.00 -1.00 
Episode 270, Step 885: -1.00 -1.00 -1.00 -1.00 -1.00 
episode reward total was -10725.000000. running mean: -9511.407339
Episode 271, Step 890: -1.00 -1.00 -1.00 -1.00 -1.00 
Episode 271, Step 895: -1.00 -1.00 -0.98 -1.00 -1.00 
episode reward total was -10780.765980. running mean: -9524.100925
Episode 272, Step 900: -1.00 -1.00 -1.00 -1.00 -1.00 
Episode 272, Step 905: -1.00 -1.00 episode reward total was -7592.000000. runnin
g mean: -9504.779916
resetting env.
   id         y
0   0  0.100000
1   6 -0.002473
2   7  0.001413
3  11  0.002787
4  12 -0.000761
5  13 -0.006929
6  14 -0.003201
7  15 -0.009545
8  16 -0.000772


g W2 -2716.28653131                                                     [4/1875]
model W2 -0.0822345222489
episode reward total was -10725.000000. running mean: -9671.336628
Episode 180, Step 890: -1.00 -1.00 -1.00 -1.00 -1.00 
Episode 180, Step 895: -1.00 -1.00 -1.00 -1.00 -1.00 
episode reward total was -10807.000000. running mean: -9682.693262
Episode 181, Step 900: -1.00 -1.00 -1.00 -1.00 -1.00 
Episode 181, Step 905: -1.00 -1.00 episode reward total was -7592.000000. runnin
g mean: -9661.786329
resetting env.
   id         y
0   0  0.100000
1   6 -0.002916
2   7 -0.000813
3  11 -0.000809
4  12 -0.004024
5  13 -0.007518
6  14 -0.002395
7  15 -0.013204
8  16 -0.003803
9  17 -0.024313


g W2 7721.04588383
model W2 -0.0834001697931
episode reward total was -10807.000000. running mean: -9812.879379
Episode 90, Step 900: -1.00 -1.00 -1.00 -1.00 -1.00 
Episode 90, Step 905: -1.00 -1.00 episode reward total was -7592.000000. running
 mean: -9790.670585
resetting env.
   id         y
0   0  0.100000
1   6  0.000197
2   7  0.000919
3  11 -0.002762
4  12  0.000973
5  13 -0.007247
6  14  0.000293
7  15 -0.013062
8  16 -0.000678
9  17 -0.024755


In [19]:
env = kagglegym.make()
observation = env.reset()
print(observation.features.head(1000))


# Submit a perfect action
target = observation.target
timestamp = observation.features["timestamp"][0]
perfect_action = df[df["timestamp"] == timestamp][["id", "y"]].reset_index(drop=True)
#observation, reward, done, info = env.step(perfect_action)
observation, reward, done, info = env.step(target)

print(reward)

while True:
    target = observation.target
    timestamp = observation.features["timestamp"][0]
    if timestamp % 200 == 0:
        print("Timestamp #{}".format(timestamp))
        print("Reward {}".format(reward))

    observation, reward, done, info = env.step(target)
    if done:        
        break
        
#info

       id  timestamp  derived_0  derived_1  derived_2  derived_3  derived_4  \
0       0        906   0.246848   0.102251   0.002781  -0.029337   0.400748   
1       7        906   0.217796   1.922894   0.752644  -0.237133   0.149876   
2      11        906  -0.074121  -0.057722   1.595148   0.311800  -0.003560   
3      12        906   0.160421  -0.037247   0.455026   0.022488   0.154796   
4      13        906  -0.025554  -0.046826        NaN  -0.026069        NaN   
5      15        906   0.069972  -0.016076        NaN  -0.113386        NaN   
6      16        906  -0.492918   0.865392   0.522874  -0.201132  -0.090198   
7      20        906  -0.010044  -0.026156  -0.006297   0.019457  -0.167557   
8      22        906   0.029458  -0.027079  -0.032081  -0.053987  -0.300417   
9      24        906  -0.049292  -0.033826   0.064581  -0.002469  -0.227432   
10     25        906  -0.077101  -0.014928   0.026619  -0.941807   0.176096   
11     26        906   0.038775  -0.030772  -0.05528

In [14]:
def extend(x, target):
    keys = list(x.keys())
    zeros = [.0] * len(keys)
    rows = x.shape[0]
    while rows < target:
        tail = pd.DataFrame(dict(zip(keys, zeros)), index=[rows])
        x = pd.concat([x, tail])
        rows = x.shape[0]
    return x

In [15]:
saver = tf.train.Saver()

init = tf.global_variables_initializer()
#init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    s = sess.run(init)
    # Restore variables from disk.
    #saver.restore(sess, "/wd/ckpt/2sigma") # comment on no checkpoint
    #print("Model restored.")
    rendering = False
    for i in range(0, 1):
        # Create environment
        env = kagglegym.make()
        # Get first observation
        observation = env.reset()

        # Reset the gradient placeholder. We will collect gradients in 
        # gradBuffer until we are ready to update our policy network. 
        gradBuffer = sess.run(tvars)
        for ix,grad in enumerate(gradBuffer):
            gradBuffer[ix] = grad * 0
        print("gradBuffer", len(gradBuffer), len(gradBuffer[0]), len(gradBuffer[1]))

        rewards = []
        done = False
        while not done:
            x = observation.features.fillna(.0) #.head(1)
            # log
            timestamp = x["timestamp"][0]
            #timestamp = 0
            debug = Debug(timestamp)
            #debug.log(x.tail(3))
            debug.log("Timestamp #{0}:{1}".format(i, timestamp))

            # Run the policy network and get an action to take. 
            y = sess.run(probability,feed_dict={observations: x})
            action = observation.target
            action["y"] = y
            # step the environment and get new measurements
            observation, reward, done, info = env.step(action)

            # get perfect action for training
            perfect_action = df[df["timestamp"] == timestamp][["id", "y"]].reset_index(drop=True)

            # calculate rewards
            pa = perfect_action
            y = action["y"]
            #debug.log("Perfect action {}".format(pa.tail(3)))
            #rewards = pa["y"] - y #.fillna(.0)
            #debug.log("(py - y).tail() {}".format(rewards.tail(3)))
            #rewards = -np.abs(rewards) #* 10
            #rewards = np.nan_to_num(rewards)
            #debug.log("rewards.shape {0}".format(rewards.shape))

            # record reward (has to be done after we call step() to get reward for previous action)
            rewards = y.shape[0] * [reward * 1.0]
            rewards = np.vstack(rewards)

            # log
            debug.log("Reward is {}".format(reward))

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(rewards)
            # size the rewards to be unit normal (helps control the gradient estimator variance)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            # Get the gradient for this episode, and save it in the gradBuffer
            py = pa["y"]
            py = py.reshape([py.shape[0],1])
            #debug.log("A={}".format(discounted_epr[0:3]))
            debug.log("y={}".format(y[100:103]))
            debug.log("py={}".format(py[100:103]))
            tGrad = sess.run(newGrads,feed_dict={observations: x, input_y: py, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                debug.log("grad-{0} is {1}".format(str(ix), grad[0]))
                gradBuffer[ix] += grad

            # If we have completed enough episodes, then update the policy network with our gradients.
            sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
            for ix,grad in enumerate(gradBuffer):
                gradBuffer[ix] = grad * 0

            if timestamp % 200 == 0:
                # Save the variables to disk.
                save_path = saver.save(sess, "/wd/ckpt/2sigma")
                print("Model saved in file: %s" % save_path)

            #if timestamp > 920:
            #    break


gradBuffer 2 110 36
Model saved in file: /wd/ckpt/2sigma
Timestamp #0:1200
Reward is -1
y=100    1.0
101    1.0
102    1.0
Name: y, dtype: float32
py=[[ 0.00679599]
 [ 0.00821976]
 [ 0.00139204]]
grad-0 is [  5.70113407e-05   2.70971977e-05  -5.09784400e-07   0.00000000e+00
   0.00000000e+00   2.35296648e-05  -3.04700116e-05   2.36279266e-05
   3.07256742e-05  -6.02099244e-05   0.00000000e+00  -3.94454291e-05
   1.04756054e-05  -1.35277933e-05   2.45801129e-05   4.69298175e-05
   0.00000000e+00   0.00000000e+00   2.41318139e-05  -4.02001569e-05
   0.00000000e+00  -3.64812549e-05  -5.17010303e-05   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
  -2.62897574e-05   5.97038525e-05   0.00000000e+00   2.98234409e-05
   2.61257683e-05   1.97549725e-05  -5.68137475e-05   0.00000000e+00]
grad-1 is [  5.36006894e-07]
Model saved in file: /wd/ckpt/2sigma
Model saved in file: /wd/ckpt/2sigma
Timestamp #0:1500
Reward is -1
y=100    1.0
101    1.0
102    1.0
Nam